In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.Builder().appName("Analysis").master("local[12]").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/08 20:22:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.parquet("../test_data.parquet")
df.show()

+--------------------+--------+------------------+-------------+--------------------+----+------------------+------------+--------------+
|                  Id|Notional|     Interest Rate|Interest Type|          Start Date|Term|Remaining Notional|Payment Type|Risk Indicator|
+--------------------+--------+------------------+-------------+--------------------+----+------------------+------------+--------------+
|14bc18bc-cb2d-458...|  285541|2.1354823813795885|        Var20|2019-01-19 15:35:...|  25|          285541.0|      Bullet|             0|
|8a597a5a-a5c7-4d6...|  763934| 3.446673585526765|        Fixed|2019-12-23 15:35:...|  20|   677580.34519278|     Annuity|             0|
|f4a94578-fa0c-45d...|  451062| 2.796471879064252|        Fixed|2016-05-20 15:35:...|  20|          300708.0|      Linear|             0|
|a0800f88-8ad8-412...|  488105|6.2589826253577705|        Var20|2016-09-14 15:35:...|  30|          488105.0|      Bullet|             4|
|5c1a3215-ce68-4f8...|  710372|2.7

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import matplotlib as mpl
from pyspark.sql import functions as fn

In [5]:
year_30 = df.filter(df['Interest Type'] == 'Var30').withColumn("nextReset", fn.add_months(fn.col('Start Date'), 12*30))
year_25 = df.filter(df['Interest Type'] == 'Var25').withColumn("nextReset", fn.add_months(fn.col('Start Date'), 12*25))
year_20 = df.filter(df['Interest Type'] == 'Var20').withColumn("nextReset", fn.add_months(fn.col('Start Date'), 12*20))
year_15 = df.filter(df['Interest Type'] == 'Var15').withColumn("nextReset", fn.add_months(fn.col('Start Date'), 12*15))
year_10 = df.filter(df['Interest Type'] == 'Var10').withColumn("nextReset", fn.add_months(fn.col('Start Date'), 12*10))
year_9 = df.filter(df['Interest Type'] == 'Var9').withColumn("nextReset", fn.add_months(fn.col('Start Date'), 12*9))
year_7 = df.filter(df['Interest Type'] == 'Var7').withColumn("nextReset", fn.add_months(fn.col('Start Date'), 12*7))
year_5 = df.filter(df['Interest Type'] == 'Var5').withColumn("nextReset", fn.add_months(fn.col('Start Date'), 12*5))

df_full = year_30.union(year_25).union(year_20).union(year_15).union(year_10).union(year_9).union(year_7).union(year_5)

In [6]:
annuities = df_full.filter(df['Payment Type'] == "Annuity")
linears = df_full.filter(df['Payment Type'] == "Linear")
bullets = df_full.filter(df['Payment Type'] == "Bullet")

In [7]:
linearpayments = linears.rdd.map(
    lambda x: (x['Id'], x['Notional'] / x['Term']))

linearpayments = linearpayments.toDF(["f_Id", "monthly_repayments"]) 
#linearpayments.show()

In [8]:
bulletpayments = bullets.rdd.map(
    lambda x: (x["Id"], 0.0)
).toDF(["f_Id", "monthly_payments"])

In [9]:
def calc_annuity_payment(notional, interest, term):
    monthsTotal = term * 12
    r = interest / 12 / 100
    payAmount = (notional*r*(pow(1+r, monthsTotal))) / (pow(1+r, monthsTotal) -1)
    return payAmount

annuitypayments = annuities.rdd.map(
    lambda x: (x["Id"], calc_annuity_payment(x["Notional"], x["Interest Rate"], x["Term"]) )
).toDF(["f_Id", "monthly_payments"])

In [10]:
#annuities.show()

In [11]:
annuities = annuities.join(annuitypayments, annuities['Id'] == annuitypayments['f_Id'], "inner")
linears = linears.join(linearpayments, linears['Id'] == linearpayments['f_Id'], "inner")
bullets = bullets.join(bulletpayments, bullets['Id'] == bulletpayments['f_Id'], "inner")
df_full = annuities.union(linears).union(bullets)

In [15]:
df_full.select("Start Date" , "nextReset", "Interest Type").collect()

[Row(Start Date=datetime.datetime(2013, 9, 28, 15, 35, 4, 82951), nextReset=datetime.date(2022, 9, 28), Interest Type='Var9'),
 Row(Start Date=datetime.datetime(2019, 12, 20, 15, 35, 3, 315688), nextReset=datetime.date(2024, 12, 20), Interest Type='Var5'),
 Row(Start Date=datetime.datetime(2019, 8, 9, 15, 35, 6, 141690), nextReset=datetime.date(2034, 8, 9), Interest Type='Var15'),
 Row(Start Date=datetime.datetime(2019, 4, 9, 15, 35, 4, 678701), nextReset=datetime.date(2024, 4, 9), Interest Type='Var5'),
 Row(Start Date=datetime.datetime(2007, 4, 10, 15, 35, 7, 513054), nextReset=datetime.date(2017, 4, 10), Interest Type='Var10'),
 Row(Start Date=datetime.datetime(2019, 1, 19, 15, 35, 5, 551933), nextReset=datetime.date(2026, 1, 19), Interest Type='Var7'),
 Row(Start Date=datetime.datetime(2017, 10, 22, 15, 35, 3, 288962), nextReset=datetime.date(2024, 10, 22), Interest Type='Var7'),
 Row(Start Date=datetime.datetime(2014, 2, 27, 15, 35, 2, 949085), nextReset=datetime.date(2034, 2, 27)